In [1]:
#### Parsing phased VCF file containing two sample for comparison


vcf_file_address = 'jupyter_phaseme/ont/compare/22_ont_10x_improved_b.vcf' #22_ont_10x.vcf'   _improved_a _g
 
vcf_file = open(vcf_file_address,'r'); 

#phased_vcf_dic={} # key: genomic_position, value: line of VCF file
hap_blocks_sample1_dic={} # key: id of phase block (i.e. phase st), value= {genomic_position:allele_1}
hap_blocks_sample2_dic={} 

#var_gpos_dic={} # key: variant index  value: list of genomic position  (with order the same as var_index)
#var_indx=0 # 1-based index for variants in the phased vcf (it will be one for the first variant)
header_lines_list=[]
for line in vcf_file:
    line_strip=line.strip() 
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:11]# last line of header contains sample name
    else:
        #var_indx+=1
          
        # '22\t51244182\t.\tG\tA\t.\t.\t.\tGT:GQ:DP:AF:GL:PS\t
        #  0|1:127:.:.:-58.12751217387621,-8.336610948353968e-14,-12.716893473415803:50415657'
        line_parts=line_strip.split('\t') 
        #phased_vcf_dic[var_indx]=line_parts     
        
        gt_flags, sample1, sample2 = line_parts[8:11]
        if "|" in sample1 and "|" in sample2: # both sample are phased for this variant
            #chr_id=line_parts[0]
            var_pos=int(line_parts[1])
            #var_gpos_dic[var_indx]=var_pos
            
            sample1_split=sample1.split(":")
            sample2_split=sample2.split(":")
            
            gt_flags_split=gt_flags.split(":")
            
            block_id1 = int(sample1_split[gt_flags.split(":").index("PS")])
            block_id2 = int(sample2_split[gt_flags.split(":").index("PS")])
            
            allele_sample1=sample1_split[gt_flags.split(":").index("GT")]
            allele_sample2=sample2_split[gt_flags.split(":").index("GT")]

            if (allele_sample1 == '0|1' or allele_sample1 == '1|0') and (allele_sample2 == '0|1' or allele_sample2 == '1|0'):
                if block_id1 in hap_blocks_sample1_dic:
                    hap_blocks_sample1_dic[block_id1][var_pos]=allele_sample1 # append new variants to the existing phase block
                else:
                    hap_blocks_sample1_dic[block_id1]={var_pos:allele_sample1} # creat new phase block

                if block_id2 in hap_blocks_sample2_dic:
                    hap_blocks_sample2_dic[block_id2][var_pos]=allele_sample2 # append new variants to the existing phase block
                else:
                    hap_blocks_sample2_dic[block_id2]={var_pos:allele_sample2} # creat new phase block


print(gt_flags_split,block_id1,block_id2,allele_sample1,allele_sample2)



# hap_blocks_sample1_dic ont
# hap_blocks_sample2_dic 10x (grand truth)


['GT', 'GQ', 'DP', 'AF', 'GL', 'PS', 'AD', 'PL', 'BX', 'PQ', 'JQ'] 50415657 41093488 0|1 0|1


In [2]:
print('Number of blocks in sample:',len(hap_blocks_sample1_dic))
#print('Number of blocks in grand truth:',len(hap_blocks_sample2_dic))

Number of blocks in sample: 21


In [3]:
for block_id in  hap_blocks_sample1_dic.keys():
    print(block_id, len(hap_blocks_sample1_dic[block_id]))
#     if len(hap_blocks_sample2_dic[block_id]) ==1:
#         print(block_id)
#         print(hap_blocks_sample2_dic[block_id])
    

16052419 548
16847870 7
16851489 75
16938155 18
16957017 9
16960113 1854
18712732 241
18873806 1816
20610824 1070
21732452 2777
24916182 3233
27896908 12
27904170 6054
37400787 176
37627246 991
39054450 4099
44814180 1174
45706994 442
46247222 1751
47886663 2786
50415657 815


In [4]:
print(len(hap_blocks_sample2_dic))

16


In [5]:
#hap_block_sample2=hap_blocks_sample2_dic[16427606]

parental_origin_blocks = {}


for block_id_sample1, hap_block_sample1 in hap_blocks_sample1_dic.items():
    
    parental_origin_block ={}
    

    for block_id_sample2, hap_block_sample2 in hap_blocks_sample2_dic.items():
        parental_origin_block_shared = []

        var_pos_list = sorted(list(hap_block_sample1.keys()))
        for var_i, var_pos in enumerate(var_pos_list):

            allele_sample1 = hap_block_sample1[var_pos]
            if var_pos in hap_block_sample2.keys():

                allele_sample2 = hap_block_sample2[var_pos]
                allele_sample2_revert = str(1-int(allele_sample2[0]))+'|'+str(1-int(allele_sample2[2]))


                if allele_sample1 == allele_sample2:
                    parental_origin = 1 #zero_one  # considering the sample1 block constant, for each grand truth, we need to change this value

                if allele_sample1 == allele_sample2_revert :
                    parental_origin = 0 #1- zero_one


                parental_origin_block_shared.append(parental_origin)
        if len(parental_origin_block_shared)>= 2:
            parental_origin_block[block_id_sample2]=parental_origin_block_shared

    
    parental_origin_blocks[block_id_sample1] = parental_origin_block
    
    
    
    
    

In [6]:
# import numpy as np
# for block_id_sample1, parental_origin_block in parental_origin_blocks.items(): 
    
#     if len(parental_origin_block)>=2: # we need to make sure that the new list starts differntly than previous
        
#         for i_block_shared in range(len(parental_origin_block)):
#             parental_origin_block_shared = np.array(parental_origin_block[i_block_shared])
#             if i_block_shared >=1:
#                     pass # we may change 
#             #print(parental_origin_block_shared[0])
#     print(block_id_sample1)


In [7]:

# import numpy as np
# # switch_all_blocks = {}
# # switch_short_blocks = {}
# switch_all = 0
# switch_short = 0

# for block_id_sample1, parental_origin_block in parental_origin_blocks.items(): 
    


    
#     for i_block_shared in range(len(parental_origin_block)):
#         parental_origin_block_shared_np = np.array(parental_origin_block[i_block_shared])

#         for i in range(len(parental_origin_block_shared_np)):
            
#             if i>=1:
#                 current = parental_origin_block_shared_np[i]
#                 pre_1  = parental_origin_block_shared_np[i-1]
                
#                 if current != pre_1: switch_all += 1
            
            
#             for length_perfect_part in range(2,21):
#                 if i >= length_perfect_part:
#                     prev_i = parental_origin_block_shared_np[i-(length_perfect_part+1)]

#                     origin_prev  = [parental_origin_block_shared_np[i] for i in range(i-length_perfect_part,i)]

#                     if (current != pre_1) and (current == prev_i) and  sum(origin_prev) == length_perfect_part*(1-current):
#                         switch_short += 1



In [8]:
# print(switch_all, switch_short)
# length_perfect_part

In [9]:

# ONT chr 19
#         long short
# raw:       25
# improved:  22

    
    
# ONT chr 22
#         long short
# raw:       79
# improved:  76
    
    
# 19_ont_10x_improved_f.vcf    
# 22_ont_10x_improved_g.vcf


In [10]:
# for block_id_sample1, parental_origin_block in parental_origin_blocks.items(): 
#         print(len(parental_origin_block))

In [11]:
# length_list= []
# for block_id_sample1, parental_origin_block in parental_origin_blocks.items(): 
#     for parental_origin_block_shared in parental_origin_block:
#         length_list.append(len(parental_origin_block_shared))
        
# print(len(length_list))

In [12]:

import numpy as np

# switch_all = 0
# switch_short = 0

list_all_consecutive= []
list_all_blocks ={}
for block_id_sample1, parental_origin_block in parental_origin_blocks.items(): 
    
    list_all_block ={}
    
    #for i_block_shared in range(len(parental_origin_block)):
        
        #parental_origin_block_shared = parental_origin_block[i_block_shared] # shared between sample (ont) and true 
    for block_id_sample2, parental_origin_block_shared in parental_origin_block.items(): 
        
        new_list=[0]
        
        for i in range(len(parental_origin_block_shared)):
            
            if i >=1:
                if parental_origin_block_shared[i] != parental_origin_block_shared[i-1]:
                    new_list.append(i)
        new_list.append(len(parental_origin_block_shared))
        list_all_consecutive.append(new_list)
        list_all_block[block_id_sample2]=new_list
    list_all_blocks[block_id_sample1]=list_all_block


In [13]:

list_length_blocks ={}
for block_id_sample1, list_all_block in list_all_blocks.items(): 

    list_length_block = {}

#     list_length_all_consecutive = []
#     list_length_all_consecutive_except_first = []

    #for list1 in list_all_block:
    for block_id_sample2, list1 in list_all_block.items(): 

        list_length = []
        for i in range(1,len(list1)):
            length1= list1[i]-list1[i-1]
            list_length.append(length1)            
#             list_length_all_consecutive.append(length1)
#             if i!=1: list_length_all_consecutive_except_first.append(length1)
#         list_length_all.append(list_length)
        list_length_block[block_id_sample2]=list_length
    list_length_blocks[block_id_sample1]=list_length_block
  

In [14]:
# list_all_blocks[20610824]

In [15]:
# list_length_blocks[20610824]

In [16]:
# len(list_length_blocks[20610824])

In [17]:
print(hap_blocks_sample1_dic.keys())
print(hap_blocks_sample2_dic.keys())

dict_keys([16052419, 16847870, 16851489, 16938155, 16957017, 16960113, 18712732, 18873806, 20610824, 21732452, 24916182, 27896908, 27904170, 37400787, 37627246, 39054450, 44814180, 45706994, 46247222, 47886663, 50415657])
dict_keys([16000159, 16123104, 16427606, 16797963, 18838130, 20326973, 20560974, 21889397, 21902136, 29079094, 29126376, 31076238, 32456138, 32726856, 35835260, 41093488])


In [18]:
for block_id in  hap_blocks_sample2_dic.keys():
    print(block_id, len(hap_blocks_sample2_dic[block_id]))

16000159 50
16123104 285
16427606 213
16797963 2110
18838130 1862
20326973 48
20560974 1106
21889397 7
21902136 6546
29079094 1
29126376 1466
31076238 707
32456138 8
32726856 2003
35835260 3730
41093488 9806


In [19]:



list_length_all = []

list_length_all_consecutive = []
list_length_all_consecutive_except_first = []

for list1 in list_all_consecutive:

    list_length = []
    for i in range(1,len(list1)):
        length1= list1[i]-list1[i-1]
        list_length.append(length1)            
        list_length_all_consecutive.append(length1)
        if i!=1: list_length_all_consecutive_except_first.append(length1)
    list_length_all.append(list_length)
            
        
        
        
    

In [20]:
len(list_length_all_consecutive_except_first)

183

In [21]:
lower_than_21=[length1<21 for length1 in list_length_all_consecutive_except_first]
greater_than_20=[length1>20 for length1 in list_length_all_consecutive_except_first]

print(sum(lower_than_21),sum(greater_than_20))





150 33


In [22]:
# import matplotlib.pyplot as plt

# plt.hist(list_length_all_consecutive)  # arguments are passed to np.histogram
# plt.title('histogram: length of perfect consecutive SNVs')
# plt.show()
